In [20]:
import openai

client = openai.OpenAI()
messages = []

In [ ]:
def get_weather(city):
    return '33 dregrees Celsius'

FUNCTION_MAP = {
    'get_weather': get_weather
}

In [24]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function that gets the weather of a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to get the weather of",
                    },
                  
                },
                "required": ["city"],
            }
        }
    }
]

In [ ]:
def call_ai():
     response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
     )
     print(response)
     message = response.choices[0].message.content
     messages.append({"role": "assistant", "content": message})
     print(f"Ai: {message}")

In [23]:
while True:
    message = input("Send a message to the LLM...")
    if message == 'quit' or message == 'q':
        break
    else :
        messages.append({"role": "user", "content": message})
        print(f"user: {message}")
        call_ai()

user: 안녕 내 이름은 기원이야
ChatCompletion(id='chatcmpl-CFiLXzqvJvqwomktZB6e91nzIcv5a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요, 기원님! 만나서 반갑습니다. 어떻게 도와드릴까요?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1757861423, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=21, prompt_tokens=64, total_tokens=85, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
Ai: 안녕하세요, 기원님! 만나서 반갑습니다. 어떻게 도와드릴까요?
user: 스페인 날씨 알려줘
ChatCompletion(id='chatcmpl-CFiLgdNbvACdrWjATHkdoS5DiQXBf', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None,

BadRequestError: Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null.", 'type': 'invalid_request_error', 'param': 'messages.[3].content', 'code': None}}